<a href="https://colab.research.google.com/github/qtuter1997/Covid-coughs/blob/main/spectrum_to_wave_data_covid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
cd /content/drive/MyDrive/Colab Notebooks/AICovidVN 115M Challenge_ Covid Cough Detection Challenge results

/content/drive/MyDrive/Colab Notebooks/AICovidVN 115M Challenge_ Covid Cough Detection Challenge results


In [19]:
pip install torchaudio

     |████████████████████████████████| 1.9MB 5.2MB/s 


In [20]:
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchaudio
import torchvision 
from tqdm import tqdm
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os
import soundfile as sf
import pandas as pd
import torch

In [21]:
cd dataset

/content/drive/MyDrive/Colab Notebooks/AICovidVN 115M Challenge_ Covid Cough Detection Challenge results/dataset


In [22]:
train_audio_file = 'aicv115m_public_train/train_audio_files_8k/'
test_audio_file = 'aicv115m_public_test/public_test_audio_files_8k/'

data_frame_train = pd.read_csv('aicv115m_public_train/data_train_new.csv')
data_frame_test = pd.read_csv('aicv115m_public_test/metadata_public_test.csv')

In [23]:
data_frame_train.head()

,Unnamed: 0,uuid,assessment_result,file_path,duration
0,0,3284bcf1-2446-4f3a-ac66-14c76b294177,0,3284bcf1-2446-4f3a-ac66-14c76b294177.wav,5.717415
1,1,431334e1-5946-4576-bb51-8e342ccc22b4,0,431334e1-5946-4576-bb51-8e342ccc22b4.wav,9.780000
2,2,1d6fac4b-1e7f-4bdc-81cd-3a720bfbb1e1,0,1d6fac4b-1e7f-4bdc-81cd-3a720bfbb1e1.wav,2.280000
3,3,c7ee0695-b2e7-4beb-b904-f1455c9609d9,0,c7ee0695-b2e7-4beb-b904-f1455c9609d9.wav,4.272517
4,4,dd541704-b696-4181-8fd8-816daac0fcf9,0,dd541704-b696-4181-8fd8-816daac0fcf9.wav,4.680000


In [24]:
data_frame_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1199 entries, 0 to 1198
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         1199 non-null   int64  
 1   uuid               1199 non-null   object 
 2   assessment_result  1199 non-null   int64  
 3   file_path          1199 non-null   object 
 4   duration           1199 non-null   float64
dtypes: float64(1), int64(2), object(2)
memory usage: 47.0+ KB


In [34]:
def wave_to_specgram(folder_path, category):    
  file_list = os.listdir(folder_path)
  dataset = torch.empty(1)
  for file in file_list:
    if file != '.DS_Store':
      waveform, sample_rate = torchaudio.load(folder_path+'/'+file)
      specgram = torchaudio.transforms.Spectrogram()(waveform)
      specgram_resize = torchvision.transforms.Resize((600,600))(specgram)
      plt.figure(frameon=False)
      plt.axis('off')
      #We add epsilon = 1e-12 in order to be able to compute the logarithm and not get NaN values in the images
      specgram_resize += torch.ones(list(specgram_resize.shape))*1e-12
      plt.imshow(specgram_resize.log2()[0,:,:].numpy(), cmap='gray')
      plt.savefig('./specgrams/' + category + '/' + folder_path.split('/')[-1] + '/'
                  + file.strip('.wav')+'.png', bbox_inches='tight',pad_inches=0)
      plt.close()

In [35]:
train_covid_path = 'aicv115m_public_train/covid'
train_healthy_path = 'aicv115m_public_train/healthy'

wave_to_specgram(train_covid_path, 'TRAIN')
wave_to_specgram(train_healthy_path, 'TRAIN')